In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, sys, inspect

# add parent dir to system dir
currdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootdir = os.path.dirname(currdir)
sys.path.insert(0, rootdir)

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [2]:
ffname_construction = os.path.join(rootdir, "data", "int", "Construction_Demographics.csv")
ffname_income = os.path.join(rootdir, "data", "int", "median_income_by_zip.csv")
ffname_homeprice = os.path.join(rootdir, "data", "int", "1_Output_quandl.csv")
ffname_mort15 = os.path.join(rootdir, "data", "int", "15_year_rates.csv")
ffname_mort30 = os.path.join(rootdir, "data", "int", "30_year_rates.csv")
ffname_schools = os.path.join(rootdir, "data", "int", "great_scools_austin.csv")
ffname_business = os.path.join(rootdir, "data", "int", "businesses_by_zip.csv")
ffname_coordinates = os.path.join(rootdir, "data", "int", "0_Lat_Long.csv")

construction_df = pd.read_csv(ffname_construction)
income_df = pd.read_csv(ffname_income)
homeprice_df = pd.read_csv(ffname_homeprice)
mortgage_15_df = pd.read_csv(ffname_mort15)
mortgage_30_df = pd.read_csv(ffname_mort30)
schools_df = pd.read_csv(ffname_schools)
business_df = pd.read_csv(ffname_business)
coordinates_df = pd.read_csv(ffname_coordinates)

In [3]:
construction_df.rename(columns={'OriginalZip': 'Zip Code', 'CalendarYearIssued': 'Year','Issuedmonth': 'Month'}, inplace=True)

In [4]:
schools_df.rename(columns={'parentRating':'School Rating (Parent)','gsRating':'School Rating(GS)'}, inplace=True)

In [5]:
homeprice_income_df = pd.merge(homeprice_df, income_df, on=["Zip Code","Year","Month"], suffixes=("_hp", "_inc"),how='left')

In [6]:
construction_homeprice_income_df = pd.merge(
    construction_df,homeprice_income_df, on=["Zip Code","Year","Month"], suffixes=("_cons", "_hpinc"),how='right')

In [7]:
mortgage_15_df.rename(columns={'year': 'Year','month': 'Month'}, inplace=True)
mortgage_30_df.rename(columns={'year': 'Year','month': 'Month'}, inplace=True)

In [8]:
mortgage_15_30 = pd.merge(mortgage_15_df, mortgage_30_df, on=["Year","Month"], suffixes=("_15", "_30"))

In [9]:
real_estate_demo_df = pd.merge(
    construction_homeprice_income_df,mortgage_15_30, on=["Year","Month"], suffixes=("_conshpinc", "_mort"),how='left')

In [10]:
real_estate_schools_df = pd.merge(
    real_estate_demo_df,schools_df, on=["Zip Code"], suffixes=("_REdemo", "_school"),how='left')

In [11]:
real_estate_business_df = pd.merge(
    real_estate_schools_df,business_df, on=["Zip Code"], suffixes=("_REschool", "_business"),how='left')
real_estate_business_df.drop(["Lat","Lng"], inplace=True, axis=1)

In [12]:
real_estate_df = pd.merge(
    real_estate_business_df,coordinates_df, on=["Zip Code"], suffixes=("_REbusiness", "_coordinates"),how='left')

In [13]:
real_estate_df = real_estate_df[["Zip Code","Year","Month","Home Value Index","Median Listing Price","Median Listing Price Per sqf","Sales",
                                 "Median Income (dollars)","Number of Households","School Rating (Parent)","School Rating(GS)","Mortgage Rate_15","Mortgage Rate_30","Total_NewConstructions",
                                 "Total_RemodelsRepairs","TotalCost_NewConstructions","TotalCost_RemodelRepairs","Bank Count","Liquor Store Count","Wal-Mart Count","Starbucks Count",
                                 "Target Count","Inventory Measure","Median Price Cut","Median Price of Reduction","Median Rental Price","Latitude","Longitude"]]

In [14]:
real_estate_df = real_estate_df.sort_values(['Zip Code', 'Year','Month'])
real_estate_df = real_estate_df.reset_index(drop=True)

In [15]:
real_estate_df.to_csv(os.path.join(rootdir, "data", "int", "realestate_demographics.csv"),index=False)